In [ ]:
import numpy as np
import json

In [2]:
class Truck:
    def __init__(self) -> None:
        self.hours_worked_day = 0
        self.hours_worked_week = 0
        self.cost_per_km = 0.68 + 20 / 100
        self.cost_per_km_overtime = 0.68 + 30 / 100
        self.packages = 0
        self.speed = 100

    def kill_trucker(self) -> None:
        pass 

class City:
    def __init__(self, req_packages:int) -> None:
        self.x = 0
        self.y = 0
        self.req_packages = req_packages
        self.remaining_packages = 0

    def recieve_packages(self, packages) -> None:
        self.remaining_packages -= packages

class Depot():
    def __init__(self) -> None:
        self.x = 0
        self.y = 0

    def make_truck(self):
        new_truck = Truck()
        return new_truck 